In [ ]:
!pip install llama-index
!pip install transformers accelerate bitsandbytes
!pip install llama-index-readers-web
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-program-openai
!pip install llama-index-agent-openai

In [ ]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

In [ ]:
!pip install InstructorEmbedding

In [ ]:
from llama_index.core import Settings
from InstructorEmbedding import INSTRUCTOR

Settings.llm = llm
Settings.embed_model='local:hkunlp/instructor-large'
embeddings = Settings.embed_model(documents)
print(embeddings)

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)


In [ ]:
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")
response = query_engine.query("What Computer Science focus areas are available to Graduate students?")

display_response(response)

**`Final Response:`** Based on the context information provided, the following Computer Science focus areas are available to graduate students:

1. Software Systems
2. Data Analytics
3. Cybersecurity
4. Non-Computer Science Background (with the option to complete 3 pre-reqs and prep for the master's program)

Therefore, the answer to the query is: Software Systems, Data Analytics, Cybersecurity, and Non-Computer Science Background.

In [ ]:
!pip install chromadb
!pip install llama_index_vector_stores_chroma

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

#initialize client
db = chromadb.PersistentClient(path="./chroma_db")

#get collection
chroma_collection = db.get_or_create_collection("quickstart")

#assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)
query_engine = vector_index.as_query_engine(response_mode="compact")
response = query_engine.query("What are Ohio CPA Examination requirements?")

display_response(response)


**`Final Response:`** Based on the context information provided, there is no mention of Ohio CPA Examination requirements. The information provided is related to Franklin University's online accounting degrees and middle childhood post-baccalaureate teacher certification program. Therefore, the answer to the query "What are Ohio CPA Examination requirements?" is "Not mentioned in the context information provided."

In [21]:
from google.colab import files
files.download('/content/chroma_db/chroma.sqlite3')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>